Tool usage

What is Tool :- Functions and services an LLM can utilize to extend its capabilities are named "tools" in Langchain.

Langchain has many tools available.

1. search tools
2. math tools
3. SqL tools

and so on...

When we decide language model to use functions there is actually 2 components to it.

1. Having language model decide which function and what the inputs to that function should be. 
2. Calling that function with those inputs.

Langchain combines these two ideas into something call tool. 

Tool is basically a schema definition for a function which then we can convert to openai function specification and callable.

We have many of these tools built into the package, search tools, math tools, SQL tools.

We will largely focus on creating our own tool.


One thing that we have noticed is that when you create you own chains and agents, a lot of it relies on actually creating your own tools because

what you are trying to do is probably pretty specific to your particular task.

1. Create own tools

2. Build a tool based on an OpenAPI spec

    Predating LLMs, the OpenAPI specification is routinely used by service providers to describe their APIs

3. Select from multiple possible tools - called routing.

In [1]:
import os
from dotenv import load_dotenv

In [ ]:
load(, ignore_discard=False, ignore_expires=False)